In [27]:
import os
import re
import markdown
import pandas as pd
from chunking_evaluation.chunking import FixedTokenChunker, RecursiveTokenChunker, ClusterSemanticChunker, LLMSemanticChunker, KamradtModifiedChunker
from chunking_evaluation import GeneralEvaluation, SyntheticEvaluation, BaseChunker
from chunking_evaluation.utils import openai_token_count

## 2. Функция препроцессинга Markdown → TXT
 
 - Конвертирует MD в HTML, затем удаляет теги.
 - Убирает нумерацию абзацев и лишние разрывы строк.
 - Сохраняет чистый текст в `.txt`.

In [28]:
def preprocess_md_to_txt(md_path: str, txt_path: str) -> None:
    with open(md_path, 'r', encoding='utf-8') as f:
        md = f.read()
    html = markdown.markdown(md)
    # Удаляем HTML-теги
    text = re.sub(r'<[^>]+>', '', html)
    # Убираем нумерацию в начале строк
    text = re.sub(r'^\s*\d+[\.\)]\s*', '', text, flags=re.MULTILINE)
    # Нормализуем пробелы и переносы
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\n{2,}', '\n\n', text)
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write(text)

## 3. Инициализация и запуск `SyntheticEvaluation`
 
Параметры:
- `approximate_excerpts`: True — гибкое извлечение, False — точное.
- `num_rounds`: число итераций генерации на документ.
- `queries_per_corpus`: вопросов за раунд (`-1` = без ограничения)

In [29]:
md_files = [
    '/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Crowl4AI/court_decisions_combined.md',
    # добавьте столько, сколько нужно
]

# Сначала преобразуем MD → TXT
txt_files = []
for md in md_files:
    txt = md.replace('.md', '.txt')
    preprocess_md_to_txt(md, txt)
    txt_files.append(txt)
    print(f"Preprocessed: {md} → {txt}")

# Путь для итогового CSV
output_csv = '/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Fine_tune_Embeddings/generated_queries.csv'

Preprocessed: /Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Crowl4AI/court_decisions_combined.md → /Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Crowl4AI/court_decisions_combined.txt


In [30]:
from dotenv import load_dotenv

# Загружаем переменные из файла .env
load_dotenv()

# Теперь переменные доступны в os.environ
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print("OpenAI API Key успешно загружен.")
else:
    print("Ошибка: OpenAI API Key не найден.")

OpenAI API Key успешно загружен.


## Запустив эту ячейку, видим все используемые системные и пользовательские промпты, на которые опирается SyntheticEvaluation.

In [31]:
from importlib import resources
import os

# Получаем путь к каталогу с промптами внутри пакета
with resources.as_file(
    resources.files('chunking_evaluation.evaluation_framework') / 'prompts'
) as prompts_dir:
    # Перечисляем все файлы промптов
    for filename in [
        'question_maker_system.txt',
        'question_maker_approx_system.txt',
        'question_maker_user.txt',
        'question_maker_approx_user.txt'
    ]:
        file_path = os.path.join(prompts_dir, filename)
        print(f"\n=== {filename} ===\n")
        with open(file_path, 'r', encoding='utf-8') as f:
            print(f.read())


=== question_maker_system.txt ===

You are an agent that generates questions from provided text. Your job is to generate a question and provide the relevant sections from the text as references.

Instructions:
1. For each provided text, generate a question that can be answered solely by the facts in the text.
2. Extract all significant facts that answer the generated question.
3. Format the response in JSON format with two fields:
   - 'question': A question directly related to these facts, ensuring it can only be answered using the references provided.
   - 'references': A list of all text sections that answer the generated question. These must be exact copies from the original text and should be whole sentences where possible.

Notes: 
Make the question more specific.
Do not ask a question about multiple topics. 
Do not ask a question with over 5 references.

Example:

Text: "Experiment A: The temperature control test showed that at higher temperatures, the reaction rate increased s

In [38]:
# 1. Указываем путь(и) к вашим текстовым корпусам и CSV
corpora_paths = [
    '/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Fine_tune_Embeddings/court_decisions_combined.txt',
]
csv_path = '/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Fine_tune_Embeddings//generated_queries_and_excerpts.csv'

## Вносим изменения в промпты, что бы генерация ответов и вопросов была на русском, так как мы хотим дообучить нашу модель на специфичном домене на русском

In [39]:
from RusSyntheticEvaluation import RusSyntheticEvaluation

In [40]:
# 1. Загрузим оригинальные английские промпты
with resources.as_file(
    resources.files('chunking_evaluation.evaluation_framework') / 'prompts'
) as prompts_dir:
    def load(name):
        with open(os.path.join(prompts_dir, name), 'r', encoding='utf-8') as f:
            return f.read()

    base_system     = load('question_maker_system.txt')
    approx_system   = load('question_maker_approx_system.txt')
    base_user       = load('question_maker_user.txt')
    approx_user     = load('question_maker_approx_user.txt')

# 2. Инструкция для генерации на русском
rus_instruction = "Please generate the question and the references in Russian.\n\n"

# 3. Собираем финальные тексты
system_prompt        = rus_instruction + base_system
approx_system_prompt = rus_instruction + approx_system

# для точного режима
user_prompt          = base_user   + "\n\n" + rus_instruction
# для приближённого режима
approx_user_prompt   = approx_user + "\n\n" + rus_instruction


# Инициализируем конвейер без указания модели
pipeline = RusSyntheticEvaluation(
    corpora_paths,
    csv_path,
    openai_api_key=openai_api_key
)
# 5. Подменяем атрибуты экземпляра
pipeline.question_maker_system_prompt        = system_prompt
pipeline.question_maker_approx_system_prompt = approx_system_prompt
pipeline.question_maker_user_prompt          = user_prompt
pipeline.question_maker_approx_user_prompt   = approx_user_prompt

# 6. Проверим, что в атрибутах действительно содержится русская инструкция
assert pipeline.question_maker_system_prompt.startswith(rus_instruction)
assert pipeline.question_maker_approx_system_prompt.startswith(rus_instruction)
assert pipeline.question_maker_user_prompt.strip().endswith(rus_instruction.strip())
assert pipeline.question_maker_approx_user_prompt.strip().endswith(rus_instruction.strip())

## теперь промпты выглядят так

In [41]:
print("=== system_prompt ===")
print(pipeline.question_maker_system_prompt[:200], "...\n")
print("=== approx_system_prompt ===")
print(pipeline.question_maker_approx_system_prompt[:200], "...\n")
print("=== user_prompt ===")
print(pipeline.question_maker_user_prompt[:200], "...\n")
print("=== approx_user_prompt ===")
print(pipeline.question_maker_approx_user_prompt[:200], "...\n")

=== system_prompt ===
Please generate the question and the references in Russian.

You are an agent that generates questions from provided text. Your job is to generate a question and provide the relevant sections from the ...

=== approx_system_prompt ===
Please generate the question and the references in Russian.

You are an agent that generates questions from provided text. Your job is to generate a question and provide the relevant sections from the ...

=== user_prompt ===
Text: {document}

The following questions have already been used. Do not repeat them: {prev_questions_str}

Do not repeat the above questions. Make your next question unique. Respond with references a ...

=== approx_user_prompt ===
Text: {document}

The following questions have already been used. Do not repeat them: {prev_questions_str}

Do not repeat the above questions. Make your next question unique. Respond with references a ...



In [42]:
# Функция для одной «пачки» генерации + фильтрации
def run_batch(batch_size=20):
    pipeline.generate_queries_and_excerpts(
        approximate_excerpts=True,
        num_rounds=1,
        queries_per_corpus=batch_size
    )
    # Удаляем «плохие» отрывки: из всех ссылок на вопрос остаются только те пары,
    # где минимальная семантическая похожесть вопроса и любого отрывка ≥ 0.36
    pipeline.filter_poor_excerpts(threshold=0.36)

    # Устраняем дубли и слишком схожие вопросы:
    # сначала убираем точные копии, затем жадным алгоритмом сбрасываем
    # все вопросы с косинусным сходством эмбеддингов > 0.7
    pipeline.filter_duplicates(threshold=0.7)

# Цикл до 200 вопросов
TARGET = 200
BATCH  = 20

# Очистим предыдущий CSV (если нужно)
if os.path.exists(csv_path):
    os.remove(csv_path)

while True:
    # Сгенерировать очередную «пачку»
    run_batch(batch_size=BATCH)
    
    # Проверить, сколько уже накопилось
    df = pd.read_csv(csv_path)
    cnt = len(df)
    print(f"Накоплено вопросов: {cnt}")
    
    if cnt >= TARGET:
        break

print(f"Готово! Всего сгенерировано и отфильтровано: {cnt} вопросов.")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [37]:
import pandas as pd
import json

# Замените на реальный путь к вашему CSV
csv_input = 'generated_queries_and_excerpts.csv'
csv_output = 'decoded_queries_Chunking_Evaluation.csv'

# 1) Загружаем
df = pd.read_csv(csv_input, dtype=str)

# 2) Функция декодирования
def decode_references(ref_str):
    try:
        data = json.loads(ref_str)
        return json.dumps(data, ensure_ascii=False)
    except json.JSONDecodeError:
        # fallback: прямой unicode-escape
        return ref_str.encode('utf-8').decode('unicode_escape')

# 3) Применяем к столбцу
df['references'] = df['references'].apply(decode_references)

# 4) Сохраняем результат
df.to_csv(csv_output, index=False, encoding='utf-8-sig')
# df.to_csv(csv_output, index=False)

print(f"Decoded table saved to {csv_output}")
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'generated_queries_and_excerpts.csv'

In [22]:
df

,question,references,corpus_id
0,Какие требования истец предъявил в суде для вз...,"[{""content"": ""использованием фотоотчетов.\nИст...",/Users/sergey/Desktop/RAG_Legal_asist/RAG_Lega...
1,Каковы были основания для восстановления срока...,"[{""content"": ""материалы дела и доводы, приведе...",/Users/sergey/Desktop/RAG_Legal_asist/RAG_Lega...
2,Каковы были основания суда для отказа ИП Безма...,"[{""content"": ""осквы от 17.10.2024 г., которым ...",/Users/sergey/Desktop/RAG_Legal_asist/RAG_Lega...
3,Какие юридические действия были предприняты фи...,"[{""content"": ""собственности на указанный объек...",/Users/sergey/Desktop/RAG_Legal_asist/RAG_Lega...
4,Какие документы представила истец Л.И. Атаманк...,"[{""content"": ""обоснование взыскания расходов н...",/Users/sergey/Desktop/RAG_Legal_asist/RAG_Lega...
...,...,...,...
179,Какие документы необходимо иметь при явке в су...,"[{""content"": ""7.\n\nПри явке в судебное заседа...",/Users/sergey/Desktop/RAG_Legal_asist/RAG_Lega...
180,Что указал истец о подписанных документах по д...,"[{""content"": ""Васкул В.Л. является застрахован...",/Users/sergey/Desktop/RAG_Legal_asist/RAG_Lega...
181,"Какова сумма общей задолженности и неустойки, ...","[{""content"": ""\""Федеральная сетевая компания Е...",/Users/sergey/Desktop/RAG_Legal_asist/RAG_Lega...
182,Какие действия по ремонту квартиры были выполн...,"[{""content"": ""2018 года по март 2021 года собс...",/Users/sergey/Desktop/RAG_Legal_asist/RAG_Lega...


In [23]:
df['references'][0]

'[{"content": "использованием фотоотчетов.\\nИстец просит суд: взыскать солидарно с ответчиков фио и Ивановой М.И. в пользу истца неосновательное обогащение полученное ответчиками в результате понесенных истцом затрат на выполнение ремонта принадлежащей ответчикам спорной квартиры - за проектную документацию в размере сумма, оплату ремонтных работ в сумме сумма и закупку строительных и отделочных материалов, инструментов, оборудования в размере сумма необходимых для ремонта квартиры принадлежащей ответчикам, а всего сумма; неосновательное обогащение полученное ответчиками в результате понесенных истцом затрат связанных с оплатой им коммунальных услуг, за отопление и содержание принадлежащей ответчикам квартиры за период с 01.02.2021 по 31.11.2021 в сумме сумма\\nСуд ", "start_index": 52294, "end_index": 53052}]'

In [24]:
import pandas as pd
import json

# Пример загрузки вашего DataFrame
df = pd.read_csv("/Users/sergey/Desktop/RAG_Legal_asist/RAG_Legal_assistant/Fine_tune_Embeddings/decoded_queries_Chunking_Evaluation .csv")  # если он из файла

def extract_contents_only(ref_string):
    try:
        # Преобразуем строку в список словарей
        refs = json.loads(ref_string)
        # Извлекаем только тексты
        return [item['content'] for item in refs if 'content' in item]
    except (json.JSONDecodeError, TypeError):
        return []

# Применяем функцию к каждой строке
df['references'] = df['references'].apply(extract_contents_only)

In [25]:
df['references'][0]

['использованием фотоотчетов.\nИстец просит суд: взыскать солидарно с ответчиков фио и Ивановой М.И. в пользу истца неосновательное обогащение полученное ответчиками в результате понесенных истцом затрат на выполнение ремонта принадлежащей ответчикам спорной квартиры - за проектную документацию в размере сумма, оплату ремонтных работ в сумме сумма и закупку строительных и отделочных материалов, инструментов, оборудования в размере сумма необходимых для ремонта квартиры принадлежащей ответчикам, а всего сумма; неосновательное обогащение полученное ответчиками в результате понесенных истцом затрат связанных с оплатой им коммунальных услуг, за отопление и содержание принадлежащей ответчикам квартиры за период с 01.02.2021 по 31.11.2021 в сумме сумма\nСуд ']

In [26]:
csv_output = 'decoded_queries_for_finetune.csv'

# 4) Сохраняем результат
df.to_csv(csv_output, index=False, encoding='utf-8-sig')